# Retrieval-Augmented Generation

## Einführung

In diesem Notebook wird eine Retrieval-Augmented Generation (RAG) für die *Magpie* implementiert. Ziel ist es, einen Chatbot zu erstellen, der Fragen in natürlicher Sprache entgegennimmt und diese in passende SQL-Abfragen umwandelt und wiederum in natürlicher Sprache antwortet.

Um die SQL-Abfragen genau auf die Daten von Magpie abzustimmen, wird auf grober Basis dieses [Tutorials](https://python.langchain.com/docs/tutorials/sql_qa/#system-prompt) und mit Langchain ein Chatbot erzeugt, der mit einem Retriever-Tool arbeitet. Der Chatbot unterscheidet sich im Vorgehen von einer *normalen* RAG: Während bei einer RAG üblicherweise Text generiert wird (die Anfrage durch einen User), der gegen eine Vektordatenbank durchsucht werden kann, besteht der Ansatz für strukturierte Daten darin, dass das LLM SQL-Abfragen schreibt und ausführt.

## Durchführung

### Arbeitspfad definieren

In einem ersten Schritt definieren wir unseren Arbeitspfad:

In [2]:
# Aktuelles Arbeitsverzeichnis ermitteln
import os
os.getcwd()
os.chdir("c:/Users/Hueck/OneDrive/Dokumente/GitHub/magpie_langchain")

### LLM laden

In einem weiteren Schritt laden wir ein LLM. Das LLM wandelt die Anfrage des Users in eine SQL-Abfrage um und antwortet basierend auf den Ergebnissen der Abfrage in natürlicher Sprache. Das sich `Mistral`, `qwen2.5:32b` und  `llama3.1:8b-instruct-q4_0` nicht so leistungsfähig wie `gpt-4o-mini` im Kontext der Generierung der SQL-Abfragen erwiesen hat, wird vorerst mit einer OpenAI-API gearbeitet. Sie wird mit `load_dotenv()` im folgenden aus dem Environment geladen um auf `gpt-4o-mini` zugreifen zu können. Der Code importiert weiterhin die `ChatOpenAI`-Klasse aus dem Modul `langchain_openai`, welches die Nutzung von OpenAI-Modellen vereinfacht. Anschließend wird ein Objekt `llm` vom Typ `ChatOpenAI` erstellt, wobei das Modell `"gpt-4o-mini"` angegeben wird.

In [3]:
from dotenv import load_dotenv

# Lade Umgebungsvariablen aus der .env Datei
load_dotenv()

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="o3-mini")

# from langchain_ollama import ChatOllama
# llm = ChatOllama(model="qwen2.5:32b", temperature=0)

### Tools genieren

Wir erstellen nun ein Set von `tools`. Das `SQLDatabaseToolkit` umfasst in diesem Kontext Werkzeuge, die uns dabei unterstützen, SQL-Abfragen zu erstellen und auszuführen sowie die Syntax von SQL-Abfragen zu überprüfen. 

Es wird dafür eine Instanz von `SQLDatabase` erstellt, um Interaktionen mit der Datenbank zu ermöglichen. Nach der Einrichtung der Datenbank wird eine Instanz von `SQLDatabaseToolkit` erstellt, die zwei Argumente benötigt: die zuvor erstellte `db`-Instanz und die oben definierte `llm` (Sprachmodell)-Instanz (siehe oben). Das Toolkit nutzt das Sprachmodell für Aufgaben wie die Validierung von Abfragen.

Die Methode `get_tools` der `SQLDatabaseToolkit`-Instanz wird dann aufgerufen, um eine Liste der verfügbaren Werkzeuge im Toolkit abzurufen. Diese Werkzeuge umfassen Funktionen wie das Auflisten von Datenbanktabellen, das Abrufen von Schema-Informationen, das Ausführen von SQL-Abfragen und das Überprüfen der Korrektheit von SQL-Abfragen. Die letzte Zeile des Codes gibt die Liste der Werkzeuge aus, sodass der Benutzer die verfügbaren Werkzeuge und deren Beschreibungen einsehen kann.

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("duckdb:///data/magpie.db")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Tool Name: sql_db_query
Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
----------------------------------------
Tool Name: sql_db_schema
Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
----------------------------------------
Tool Name: sql_db_list_tables
Description: Input is an empty string, output is a comma-separated list of tables in the database.
----------------------------------------
Tool Name: sql_db_query_checker
Description: Use this tool to double check if your query is

d:\Programme\miniconda\envs\chatbot_magpie\lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


### Retriever Tool

Im folgenden wird ein Retriever-Tool erzeugt, dass dabei hilft, *fehlerhafte* Abfragen seitens der User zu korrigieren. Das Retriever Tool versteht so z.B. dass "*"Frei Uni Nerlin"*" für "*Freie Universität Berlin*" steht.

Zunächst werden die benötigten Bibliotheken importiert. LangChain-Module werden verwendet, um Embeddings und einen In-Memory-Vektorspeicher zu erstellen. Zudem werden `ast` und `re` für die Verarbeitung und Bereinigung von Daten genutzt.

Die Funktion `query_as_list(db, query)` führt eine SQL-Abfrage auf Magpie `db` aus. Das Ergebnis wird zunächst in eine Liste umgewandelt, dabei werden verschachtelte Strukturen aufgelöst. Danach werden alle Zahlen aus den Ergebnissen entfernt, und Leerzeichen werden bereinigt. Schließlich wird sichergestellt, dass nur eindeutige Werte in der Liste enthalten sind.

Anschließend werden mit `query_as_list` zwei Listen erstellt:  
- `Variable`, die alle Werte aus der Spalte *Variable* der Tabelle `datensatz_drittmittel_hochschule` enthält.  
- `Hochschule`, die alle Werte aus der Spalte *Hochschule* derselben Tabelle enthält.  

Nun wird ein Embedding-Modell von OpenAI mit dem Namen `"text-embedding-3-large"` initialisiert. Dieses Modell wandelt Texte in numerische Vektoren um, die für eine semantische Suche genutzt werden können.

Ein In-Memory-Vektorspeicher wird erstellt, der diese Vektoren speichert. Anschließend werden die Begriffe aus `Variable` und `Hochschule` als Texte in den Vektorspeicher hinzugefügt.

Der Vektorspeicher wird nun als Retriever konfiguriert. Der Parameter `search_kwargs={"k": 5}` bedeutet, dass für jede Suchanfrage die fünf ähnlichsten Einträge zurückgegeben werden.

Zuletzt wird mit `create_retriever_tool` ein Retriever-Tool erstellt. Dieses Tool kann verwendet werden, um nach ähnlichen Namen oder Begriffen zu suchen. Die Beschreibung des Tools gibt an, dass es dazu dient, gültige Namen zu finden, die am besten zu einer unscharfen Eingabe passen.

In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT beschr FROM variable")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(beschr_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_beschr_variable = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [ ]:
#retr_tool = retriever_tool.invoke("Studienanfänger ohdfne Abitur")
output = rt_beschr_variable.invoke("Technisches Personal")
print(output)

Technisches FuE-Personal

Wissenschaftliches FuE-Personal

Sonstiges FuE-Personal

FuE-Personal

FuE-Personal ohne Wissenschaftler


In [39]:
reichweite_variable = query_as_list(db, "SELECT beschr FROM Reichweite")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = InMemoryVectorStore(embeddings)

_ = vector_store.add_texts(reichweite_variable)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

description = (
    "Use to look up values to filter on. Input is an approximate spelling "
    "of the proper noun, output is valid proper nouns. Use the noun most "
    "similar to the search."
)

rt_reichweite_variable = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description,
)

In [ ]:
#retr_tool = retriever_tool.invoke("Studienanfänger ohdfne Abitur")
output = rt_reichweite_variable.invoke("FU Berlin")
print(output)

In [20]:
from langchain_core.tools import tool

@tool
def get_variable_id(description: str) -> str:
    """
    Sucht in der Tabelle 'variable' nach einer passenden ID basierend auf der Beschreibung.
    Nutzt das bestehende Retrieval-Tool, um Tippfehler oder ungenaue Eingaben zu korrigieren.
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = retriever_tool.invoke(description)
    print(matches)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Nimm das relevanteste Ergebnis, also das. 1.

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT id FROM variable WHERE beschr = '{best_match}' LIMIT 1;"
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."


@tool
def get_reichweite_id(description: str) -> str:
    """
    Sucht in der Tabelle 'reichweite' nach einer passenden ID basierend auf der Beschreibung.
    Nutzt das bestehende Retrieval-Tool, um Tippfehler oder ungenaue Eingaben zu korrigieren.
    """
    # Verwende das bestehende Retriever-Tool, um die beste Übereinstimmung zu finden
    matches = retriever_tool.invoke(description)
    if not matches:
        return "Error: Keine passende Variable gefunden."

    best_match = matches.split("\n")[0]  # Nimm das relevanteste Ergebnis, also das. 1.

    # SQL-Abfrage mit der gefundenen besten Übereinstimmung
    query = f"SELECT id FROM Reichweite WHERE beschr = '{best_match}' LIMIT 1;"
    print(query)
    result = db.run_no_throw(query)

    return result if result else "Error: Keine passende Variable gefunden."

@tool
def get_wert_einheit_id(description: str) -> str:
    """Sucht in der Tabelle 'wert_einheit' nach einer passenden ID basierend auf der Beschreibung."""
    query = f"SELECT id FROM wert_einheit WHERE beschr LIKE '%{description}%' LIMIT 1;"
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passende Werteinheit gefunden."

@tool
def get_data(variable_id: str, reichweite_id: str, wert_einheit_id: str) -> str:
    """Extrahiert Daten aus der Tabelle 'daten' basierend auf den IDs der Variable, Reichweite und Werteinheit."""
    query = f"""
        SELECT wert, jahr 
        FROM daten 
        WHERE id = '{variable_id}' 
        AND id = '{reichweite_id}' 
        AND id = '{wert_einheit_id}'
        ORDER BY jahr;
    """
    result = db.run_no_throw(query)
    return result if result else "Error: Keine passenden Daten gefunden."

tools.extend([get_variable_id, get_reichweite_id, get_wert_einheit_id, get_data])


In [22]:
#print(get_variable_id.invoke("Studienanfänger ohdfne Abitur"))
print(get_reichweite_id.invoke("Fachhochsdfchulen"))
# print(get_wert_einheit_id.invoke("Anzahl"))
#print(get_data.invoke("1", "2", "3"))  

#db.run_no_throw("SELECT id FROM variable WHERE beschr = 'Anzahl der Studienanfänger ohne Abitur' LIMIT 1;")

SELECT id FROM Reichweite WHERE beschr = 'Fachhochschule' LIMIT 1;
[(8,)]


### Prompt Template generieren 

Im folgenden wird das `langchain`-Paket verwendet, um ein Prompt-Template von `langchain-ai` zu laden und zu überprüfen. Zuerst wird die `hub`-Funktion aus dem `langchain`-Paket importiert, um auf Modelle und Vorlagen im LangChain Hub zuzugreifen. Mit `hub.pull("langchain-ai/sql-agent-system-prompt")` wird ein spezifisches Prompt-Template abgerufen, in diesem Fall das `sql-agent-system-prompt`. `sql-agent-system-prompt` ist ein vordefiniertes Prompt-Template. Es dient als Vorlage für die Interaktion mit einer SQL-Datenbank über ein Sprachmodell. Dieses Template ist speziell für die Verwendung mit SQL-Agenten konzipiert, die SQL-Abfragen generieren oder validieren können.

Der Code prüft weiterhin mit `assert len(prompt_template.messages) == 1`, ob das geladene Template genau eine Nachricht enthält. Falls dies nicht zutrifft, wird ein Abbruch ausgelöst. Abschließend wird mit `prompt_template.messages[0].pretty_print()` die Nachricht im Template in einem gut lesbaren Format ausgegeben, um sicherzustellen, dass der Inhalt korrekt geladen wurde.

In [25]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

d:\Programme\miniconda\envs\chatbot_magpie\lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to th

Wir befüllen nun die Objekte, die in `prompt_template` bisher nur als Platzhalter definiert sind. Diese sind:
1. `dialect`
2. `top_k`

 `dialect` meint den SQL-Dialekt der verwendeten Datenbank. `top_k` bestimmt die Anzahl der zurückgegebenen, *besten Ergebnisse*. In diesem Kontext bedeutet *beste Ergebnisse* die Auswahl der relevantesten oder nützlichsten Ergebnisse der Abfragen, etwa bzgl. der Relevanz im Zusammenhang mit einer Abfrage.


In [29]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct duckdb query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see w

### Agent generieren    

Es wird nun eine Instanz eines *React-Agenten* aus der `langchain_core` und `langgraph` Bibliothek erstellt und konfiguriert. Im einzelnen passiert das Folgende: Die Funktion `create_react_agent(llm, tools, state_modifier=system_message)` erstellt einen neuen Agenten, der auf das bereitgestellte Sprachmodell (`llm`) und eine Sammlung von Werkzeugen (`tools`) zugreifen kann, um Aufgaben zu erledigen. Das Sprachmodell (`llm`) ist die Instanz eines Modells `gpt-4o-mini`, das vom Agenten verwendet wird, während tools die Werkzeuge sind, mit denen der Agent auf Daten zugreifen oder Aktionen durchführen kann, zum Beispiel eine SQL-Datenbankabfrage oder APIs. `state_modifier=system_message` gibt eine Systemnachricht an, die die Rolle des Agenten und seine Aufgaben innerhalb des Systems beschreibt, einschließlich der Anweisungen, wie der Agent mit den Eingaben des Benutzers umgehen soll. Die erstellte Instanz des React-Agenten wird in der Variablen `agent_executor` gespeichert, die dann verwendet wird, um Interaktionen mit einem Benutzer zu führen und die entsprechenden Werkzeuge anzuwenden.

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Schritt 1: Alle Tools zum Agenten hinzufügen
tools.extend([retriever_tool, get_variable_id, get_reichweite_id, get_data])

# Systemnachricht mit extra Anweisungen
suffix = (
    "Before generating an SQL query, you must first: \n"
    "1. Use 'search_proper_nouns' to correct user input.\n"
    "2. Use 'get_variable_id' to get the correct variable ID.\n"
    "3. Use 'get_reichweite_id' to get the correct range ID.\n"
    "4. Use 'get_data' to retrieve the relevant data.\n"
    "NEVER attempt to guess values—always use these tools."
)

system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)



================================ Human Message =================================

Wie hoch waren die Drittmittel vom Bund von der Uni Kassel im 2006?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_DOOImCviRfQurp8HEn1ymDra)
 Call ID: call_DOOImCviRfQurp8HEn1ymDra
  Args:
    tool_input:
================================= Tool Message =================================
Name: sql_db_list_tables

antwort, chart_options_rules, daten, daten_frage, daten_quelle, daten_reichweite, datensatz_drittmittel_aggregiert, datensatz_drittmittel_hochschule, datensatz_fue_erhebung, datensatz_hbr_report, erhebung, erhebung_frage, erhebung_typ, erklaerung, erklaerung_link, frage, germany_choropleth_federal_states, glossar, kurse, menge, menge_zugehoerig, middle_points_of_ger_federal_states, mview_daten_beschr, mview_daten_id, mview_daten_reichweite_menge, quelle, reichweite, reichweite_klasse, reichweite_typ, stichprobe, studierende, st

## Probelauf Chatbot

Schließlich testen wir den Bot: Zunächst wird dafür eine Frage (`question`) definiert: *"Wie hoch waren die Drittmittel der FU Berlin im Jahr 2006 insgesamt?"*. Diese Frage wird später an den Agenten übergeben.

Dann wird eine Schleife (`for step in agent.stream(...)`) gestartet, die den Agenten Schritt für Schritt durch den Verarbeitungsprozess führt.  

- Der Agent erhält die Eingabe als eine Nachricht im Format `{"role": "user", "content": question}`.  
- Das Argument `stream_mode="values"` sorgt dafür, dass die Antwort in einzelnen Schritten ausgegeben wird.  
- Innerhalb der Schleife wird der jeweils letzte Schritt (`step["messages"][-1]`) formatiert und ausgegeben, sodass die Antwort für den Nutzer lesbar bleibt.  

Dadurch wird der Verarbeitungsweg der Antwort des Agenten schrittweise angezeigt.

In [28]:
# Testanfrage an den Agenten
question = "Studienanfänger 2019 ohne Abitur?"

for step in agent_executor.stream(
    {"messages": [HumanMessage(content=question)]}, 
    stream_mode="values"
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Studienanfänger 2019 ohne Abitur?
================================== Ai Message ==================================
Tool Calls:
  search_proper_nouns (call_U7wIMGeRqxYVRXehbszQa01b)
 Call ID: call_U7wIMGeRqxYVRXehbszQa01b
  Args:
    query: Studienanfänger 2019 ohne Abitur
================================= Tool Message =================================
Name: search_proper_nouns

Studienabsolventen ohne Abitur

Anzahl der Studienanfänger ohne Abitur

Anteil der Studienanfänger ohne Abitur

Anteil der Studienabsolventen ohne Abitur

Anzahl der Studienanfänger duales Studium
================================== Ai Message ==================================
Tool Calls:
  get_variable_id (call_I7LlWS3a2JdCDg9IPO2SFPI8)
 Call ID: call_I7LlWS3a2JdCDg9IPO2SFPI8
  Args:
    description: Anzahl der Studienanfänger ohne Abitur
Anzahl der Studienanfänger ohne Abitur

Anteil der Studienanfänger ohne Abitur

Anteil der St